In [15]:
import polars as pl
from datetime import date

In [2]:
FILE_PATH = 'AllChains Data.csv'

In [202]:
MAX_DATE = date(2023, 6, 30)
MAX_MONTH_FIRST_DATE = date(2023, 6, 1)

In [191]:
df = pl.read_csv(FILE_PATH)
print(df)

shape: (8_090, 5)
┌────────────┬─────────────┬───────────┬─────────┬──────────┐
│ Date       ┆ Store       ┆ Bike Type ┆ Sales   ┆ Profit   │
│ ---        ┆ ---         ┆ ---       ┆ ---     ┆ ---      │
│ str        ┆ str         ┆ str       ┆ f64     ┆ f64      │
╞════════════╪═════════════╪═══════════╪═════════╪══════════╡
│ 03/01/2020 ┆ Brixton     ┆ Gravel    ┆ 16.448  ┆ 5.5512   │
│ 04/01/2020 ┆ Westminster ┆ Mountain  ┆ 3.54    ┆ -5.487   │
│ 04/01/2020 ┆ Whitechapel ┆ Mountain  ┆ 11.784  ┆ 4.2717   │
│ 04/01/2020 ┆ Peckham     ┆ Mountain  ┆ 272.736 ┆ -64.7748 │
│ 05/01/2020 ┆ Chelsea     ┆ Gravel    ┆ 19.536  ┆ 4.884    │
│ …          ┆ …           ┆ …         ┆ …       ┆ …        │
│ 30/12/2023 ┆ Westminster ┆ Gravel    ┆ 55.8    ┆ 19.1862  │
│ 30/12/2023 ┆ Westminster ┆ Road      ┆ 34.624  ┆ 10.9938  │
│ 30/12/2023 ┆ Shoreditch  ┆ Gravel    ┆ 323.136 ┆ 12.1176  │
│ 30/12/2023 ┆ Mayfair     ┆ Gravel    ┆ 3.024   ┆ -0.6048  │
│ 30/12/2023 ┆ Richmond    ┆ Gravel    ┆ 90.93   ┆ 2

In [288]:
intermediate = (
    df
    .with_columns(
        pl.col('Date').str.to_date('%d/%m/%Y'),
    )
    .filter(pl.col('Date').le(MAX_DATE))
    .groupby_dynamic('Date', every='1mo', by=['Store', 'Bike Type']).agg(
        pl.col('Sales').sum(), 
        pl.col('Profit').sum()
    )
)

supplimental_rows = (
    intermediate
    .with_columns(
        pl.col('Date').max().over(['Store', 'Bike Type']).alias('max month per group')
    )
    .filter(
        (pl.col('max month per group')!=MAX_MONTH_FIRST_DATE)
        &
        (pl.col('max month per group')==pl.col('Date'))
    )
    .select(
        'Store',
        'Bike Type',
        pl.lit(MAX_MONTH_FIRST_DATE).alias('Date'),
        pl.lit(0.0).alias('Sales'),
        pl.lit(0.0).alias('Profit')
    )
)


result = (
    intermediate
    .vstack(supplimental_rows)
    .sort('Store', 'Bike Type', 'Date')
    .set_sorted(['Date', 'Store', 'Bike Type'])
    .upsample('Date', every='1mo', by=['Store', 'Bike Type'])  # filling dates where there is no sales value
    .with_columns(
        pl.col(['Store', 'Bike Type']).fill_null(strategy='forward'),
        pl.col(['Sales', 'Profit']).fill_null(0)
    )
    .select(
        pl.col('Date').alias('Month'),
        pl.col('Store'),
        pl.col('Bike Type'),
        pl.col('Sales').round(2),
        pl.col('Profit').round(2),
        pl.col('Profit').rolling_mean(window_size=3).over(['Store', 'Bike Type']).round(2).alias('3 Month Moving Average Profit')
    )
    .sort(['Store', 'Bike Type', 'Month'])
)
result

Month,Store,Bike Type,Sales,Profit,3 Month Moving Average Profit
date,str,str,f64,f64,f64
2020-01-01,"""Brixton""","""Gravel""",199.36,80.88,null
2020-02-01,"""Brixton""","""Gravel""",91.29,34.19,null
2020-03-01,"""Brixton""","""Gravel""",453.84,196.16,103.74
2020-04-01,"""Brixton""","""Gravel""",190.1,80.34,103.56
2020-05-01,"""Brixton""","""Gravel""",242.55,113.74,130.08
…,…,…,…,…,…
2023-02-01,"""Wimbledon""","""Road""",1429.33,261.57,153.94
2023-03-01,"""Wimbledon""","""Road""",1527.74,347.37,199.98
2023-04-01,"""Wimbledon""","""Road""",146.31,15.6,208.18
